In [17]:
import pandas as pd
import numpy as np
import mlflow
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

In [3]:
data="data/wine-quality.csv"

In [4]:
data=pd.read_csv(data)

In [6]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
data.shape

(4898, 12)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [9]:
labels=data.pop("quality")

In [12]:
X_train,X_test,y_train,y_test=train_test_split(data,labels,random_state=42)

In [13]:
remote_server_uri="http://127.0.0.1:5000"
mlflow.set_tracking_uri(remote_server_uri)

In [14]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [15]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/06/01 16:24:19 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [18]:
def eval_metrics(y,y_pred):
    rmse=np.sqrt(mean_squared_error(y,y_pred))
    mae=mean_absolute_error(y,y_pred)
    r2=r2_score(y,y_pred)
    return rmse,mae,r2

def train(alpha=0.5,l1_ratio=0.5):
    data=pd.read_csv('data/wine-quality.csv')
    labels=data.pop('quality')
    X_train,X_test,y_train,y_test=train_test_split(data,labels,random_state=42)
    
    with mlflow.start_run():
        model=ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        model.fit(X_train,y_train)
        y_pred=model.predict(X_test)
        rmse,mae,r2=eval_metrics(y_test,y_pred)
        mlflow.log_param("alpha",alpha)
        mlflow.log_param("l1_ratio",l1_ratio)
        mlflow.log_metrics({"rmse":rmse,"mae":mae,"r2":r2})
        mlflow.log_artifact('data/wine-quality.csv')
        mlflow.sklearn.log_model(model, "model")

In [19]:
train()

/home/sudarshan.magaji/miniconda3/envs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [20]:
train(alpha=0.4,l1_ratio=0.8)

In [21]:
train(alpha=0.2,l1_ratio=0.8)

In [22]:
from  mlflow.tracking import MlflowClient

In [23]:
client = MlflowClient()

In [24]:
experiments = client.list_experiments()

In [25]:
experiments

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>]